<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [137]:
# Импорт необходимых библиотек
import pandas as pd
import psycopg2
# Импорт игнорирования ошибки
import warnings
warnings.filterwarnings('ignore')

In [138]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [139]:
# текст запроса
query_3_1 = f'''SELECT COUNT(id) AS "Количество ваканский"
FROM vacancies
'''

In [140]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,Количество ваканский
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [141]:
# текст запроса
query_3_2 = f'''SELECT COUNT(id) AS "Количество работодателей"
FROM employers
'''

In [142]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,Количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [143]:
# текст запроса
query_3_3 = f'''SELECT COUNT(id) AS "Количество регионов"
FROM areas
'''

In [144]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,Количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [145]:
# текст запроса
query_3_4 = f'''SELECT COUNT(id) AS "Количество сфер деятельности"
FROM industries
'''

In [146]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,Количество сфер деятельности
0,294


***

Предварительный анализ помог ознакомиться с данными в таблицах. Запросы позволили определить количество важных для анализа объектов в схеме.

Итого:

1. Всего вакансий: 49197
2. Всего работодателей: 23501
3. Всего регионов (населенных пунктов): 1362
4. Всего сфер деятельности: 294

Предварительно, зная только эту информацию и учитывая связи между данными по схеме, можно провести более детальный анализ данных.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [147]:
# текст запроса
query_4_1 = f'''SELECT
    a.name AS "Регион (населенный пункт)",
    COUNT(*) AS "Количество вакансий"
FROM areas a
JOIN vacancies v ON v.area_id = a.id --Приссоединяем к таблице регионов таблицу вакансий
GROUP BY a.name
ORDER BY 2 DESC
LIMIT 5
'''

In [148]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,Регион (населенный пункт),Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [149]:
# текст запроса
query_4_2 = f'''SELECT
    COUNT(id) AS "Количества вакансий с з/п"
FROM vacancies 
WHERE salary_from is not null -- Фильтр для верхней и нижней границ з/п
OR salary_to is not null
'''

In [150]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,Количества вакансий с з/п
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [151]:
# текст запроса
query_4_3 = f'''SELECT
    ROUND(AVG(salary_from)) AS "Нижняя граница зарплатной вилки", -- Среднее для нижней границы з/п
    ROUND(AVG(salary_to)) AS "Верхняя граница зарплатной вилки"   -- Среднее для верхней границы з/п
FROM vacancies
'''

In [152]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,Нижняя граница зарплатной вилки,Верхняя граница зарплатной вилки
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [153]:
# текст запроса
query_4_4 = f'''SELECT
    schedule AS "Тип рабочего графика",
    employment AS "Тип трудоустройства",
    COUNT(*) AS "Количество"
FROM vacancies
GROUP BY 1, 2
ORDER BY 3 DESC
'''

In [154]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,Тип рабочего графика,Тип трудоустройства,Количество
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [155]:
# текст запроса
query_4_5 = f'''SELECT
    experience AS "Требуемый опыт работы",
    COUNT(*) AS "Количество"
FROM vacancies
GROUP BY 1
ORDER BY 2
'''

In [156]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,Требуемый опыт работы,Количество
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

1. Количество предалагаемых вакансий напрмую зависит от площади и населения региона. Чем крупнее населенный пункт, тем больше вакансий предлагается в нем.
2. Половина вакансий не представлена одной из границ ЗП, поэтому возможно предположить, что данные вакансии имеют определеную фиксированную ставку или указаны одной суммой "от" или "до".
3. Средне-максимальная ЗП выше средне-минимальной в полтора раза (110537.0 / 71065.0), что свидетельствует о разбросе сумм по оплате труда и может зависеть от региона, в котором представлены эти вакансии.
4. Очевидно, что работодателям требуются сотрудники на полный рабочий день с полной занятостью, но также стоит отметить, что и вакансии  удаленной работы тоже популярны, потому что имеется опыт работ компаний с предоставлением такого типа занятости в период карантина.
5. По опыту работы работодатель в целом готов рассматривать кандидатов с имеемым небольшим опытом работы от 1 года до 3 лет. Стоит отметить, что наверное имеет смысл проанализировать данные по требуемому опыту работы в разрезе ЗП.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [157]:
# текст запроса
query_5_1 = f'''SELECT
    e.name AS "Работодатели",
    COUNT(*) AS "Количество"
FROM vacancies v
JOIN employers e ON e.id = v.employer_id -- Приссоединяем к таблице вакансии таблицу работодатели
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 5
'''

In [158]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,Работодатели,Количество
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [159]:
# текст запроса
query_5_2 = f'''SELECT
    a.name AS "Регион (населенный пункт)",
    COUNT(e.id) AS "Количество работодателей",
    COUNT(v.id) AS "Количество вакансий"
FROM areas a 
JOIN employers e ON a.id = e.area -- Приссоединяем к таблице регионов таблицу работодатели
LEFT JOIN vacancies v ON a.id = v.area_id -- Затем слева приссоединяем таблицу вакансии
WHERE v.id IS NULL  -- Фильтр регионов, в которых нет вакансий
GROUP BY 1
ORDER BY 3, 2 DESC
LIMIT 5
'''

In [160]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,Регион (населенный пункт),Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [161]:
# текст запроса
query_5_3 = f'''SELECT
    e.name AS "Работодатели",
    COUNT(DISTINCT v.area_id) AS "Количество регионов"
FROM vacancies v
JOIN employers e ON v.employer_id = e.id -- Приссоединяем к таблице вакансии таблицу работодатели
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
'''

In [162]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,Работодатели,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [163]:
# текст запроса
query_5_4 = f'''SELECT 
    COUNT(*)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id -- Связь между работодателями и сферами их деятельности
WHERE ei.employer_id IS NULL -- Фильтр работодателей, у которых не указана сфера деятельности
'''

In [164]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [165]:
# текст запроса
query_5_5 = f'''SELECT 
    e.name AS "Работодатели"
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id -- Связь между работодателями и сферами их деятельности
GROUP BY e.name -- Группируем по имени компаний
HAVING COUNT(DISTINCT ei.industry_id) = 4 -- Фильтр компаний, у которых указано 4 сферы деятельности
ORDER BY e.name
LIMIT 3
'''

In [166]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,Работодатели
0,101 Интернет
1,21vek.by
2,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [167]:
# текст запроса
query_5_6 = f'''SELECT 
    COUNT(*)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id -- Связь между работодателями и сферами их деятельности
JOIN industries i ON i.id = ei.industry_id -- Приссоединяем таблицу сферы деятельности к таблице работодатели
WHERE i.name = 'Разработка программного обеспечения' -- Фильтр по условию
'''

In [168]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [169]:
# код для получения списка городов-милионников
# Формируем список регионов-миллионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
tables = pd.read_html(url)
city_df = tables[1]
city_list = list(city_df['Город'])
city_list[0] = 'Москва'
display(city_list)

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [170]:
# текст запроса
query_5_7 = f'''SELECT
        a.name AS "Регионы-миллионники",
        COUNT(v.id) AS "Количество вакансий"
    FROM vacancies v
    -- Приссоединяем к таблице вакансии таблицы работодатели и регионы
    LEFT JOIN employers e on v.employer_id = e.id 
    LEFT JOIN areas a on v.area_id = a.id
    WHERE e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 
    'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
    GROUP BY a.name

    UNION ALL -- Объединение таблиц

    SELECT 
        'Total', 
        COUNT(*)
    FROM vacancies v
    -- Приссоединяем к таблице вакансии таблицы работодатели и регионы
    LEFT JOIN employers e on v.employer_id = e.id
    LEFT JOIN areas a on v.area_id = a.id
    WHERE e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 
    'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
    ORDER BY 2
'''

In [171]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,Регионы-миллионники,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

1. Компания "Яндекс" является значительным лидером среди работодателей по количеству предложенных вакансий, опережая конккурента почти в 4 раза.
2. Определили регионы, в которых нет вакансий, но есть работодатели. Этими регионами являются либо названия государств (Россия, Казахстан, Беларусь), либо названия субъектов РФ (Московская область, Краснодарский край и т.п.). Кличество неуказанных ваканский почти не касается именно городов и значительно падает после показателей в Казахстане. Но этого необходимо учитывать при дальнейшем более детальном и глубоком анализе данных, т.к. этот факт может влиять на качество анализируемых нами данных.
3. Наибольшее количество охватываемых регионов ожидаемо у компании "Яндекс", т.к. является лидером по количеству вакансий во всей БД (около 4% от всего количества).
4. Выявлено 8419 работодателей, у которых не указана сфера деятельности. Этот показатель необходимо будет учитывать при дальнейшем анализе данных.
5. При необходимости можем определить работодателей, которые указывают несколько сфер деятельности, в зависимости от потребнеостей проводимого анализа данных.
6. Установлено, что сфера деятельности "Разработка программного обеспечения" является довольно востребованной специальностью и этот показатель составляет 7% от общего количества вакансий в БД.
7. С учетом лидерства компании "Яндекс" целесобразно было провести анализ количества вакансий по регионам-миллионникам. Ожидаемо, что лидерами реди регионов являются самые большие города России - Москва и Санкт-Петербург.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [172]:
# текст запроса
query_6_1 = f'''SELECT
    COUNT(v.name)
FROM vacancies v
-- Фильтр по условиям
WHERE lower(v.name) LIKE '%data%'
    OR lower(v.name) LIKE '%данн%'
'''

In [173]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [174]:
# текст запроса
query_6_2 = f'''SELECT
    COUNT(v.name)
FROM vacancies v
-- Фильтр по условиям
WHERE (
    lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    ) AND
    (lower(v.name) LIKE '%junior%'
    OR v.experience = 'Нет опыта'
    OR v.employment = 'Стажировка'
    )
'''

In [175]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [176]:
# текст запроса
query_6_3 = f'''SELECT
    COUNT(v.name)
FROM vacancies v
-- Фильтр по условиям
WHERE (
    lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    ) AND
    (upper(v.key_skills) LIKE '%SQL%'
    OR lower(v.key_skills) LIKE '%postgres%'
    )
'''

In [177]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [178]:
# текст запроса
query_6_4 = f'''SELECT
    COUNT(v.name)
FROM vacancies v
-- Фильтр по условиям
WHERE (
    lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    ) AND
    (lower(v.key_skills) LIKE '%python%')
'''

In [179]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [180]:
# текст запроса
# Рассчет среднего значения
query_6_5 = f'''SELECT
    ROUND(AVG(LENGTH(v.key_skills) -
    LENGTH(REPLACE(v.key_skills, chr(9), '')) + 1), 2) AS "Среднее значение вакансий для DS"
FROM vacancies v
-- Фильтр по условиям
WHERE (
    lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
'''

In [181]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,Среднее значение вакансий для DS
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [182]:
# текст запроса
# Рассчет среднего значения
query_6_6 = f'''SELECT
    ROUND(AVG(COALESCE((v.salary_to + v.salary_from)/2, v.salary_to, v.salary_from))) AS
    "Средняя з/п для DS"
FROM vacancies v
-- Фильтр по условиям
WHERE (
    lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND (v.salary_to IS NOT NULL OR v.salary_from IS NOT NULL)
    AND v.experience = 'От 3 до 6 лет'
'''

In [183]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,Средняя з/п для DS
0,243115.0


1. Количество вакансий, связанных c DS, - 1771, что составляет около 3,6 % от общего числа вакансий в БД.
2. Определили количество вакансий для новичков в DS. Оказалось, что вакансий не так много, всего 0,1 % от обего количества вакансий. Но тем не менее такие вакансии представлены и поэтому есть возможность трудоустройства сразу после обучения или даже в период прохождения курсов.
3. Определили, что такие ключевые навыки как знания SQL и postgres необходимы для специалиста по DS, т.к. это 11 % от общего количества вакансий, связанных c DS.
4. Знания языка программирования Python имеют еще большую востребованность среди соискателей вакансий, связанных c DS, и составляют почти 20 % от общего числа вакансий DS.
5. Установлено, что в среднем работодатели указывают порядка 6-7 ключевых навыков для сосискателей DS. Это могут быть и hard skills, так и smart skills.
6. Вывели один из самых важных критериев для соискателей работы в области DS - среднюю з/п для специалистов с опытом работы от 3 до 6 лет, которая составляет около 243 тыс. 115 рублей.

# Юнит 7. Дополнительные исследования данных

1. На первом этапе дополнительных исследований выведем ТОП-10 представленных в БД вакансий.

In [193]:
# текст запроса
query_7_1 = f'''SELECT
    v.name AS "Название вакансии",
    COUNT(v.id)
FROM vacancies v
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
'''

In [194]:
# результат запроса
df = pd.read_sql_query(query_7_1, connection)
df

,Название вакансии,count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


2. И так сразу можно сделать вывод, что вакансии в сфере ИТ очень востребованы. Поэтому проведем анализ количества вакансий по сферам деятельности и выведем ТОП-10 в привязке по количеству вакансий.

In [195]:
# текст запроса
query_7_2 = f'''SELECT
    i.name AS "Сфера деятельности",
    COUNT(v.id) AS "Количество вакансий"
FROM industries i
-- Связь между работодателями и сферами их деятельности
LEFT JOIN employers_industries ei ON i.id = ei.industry_id
LEFT JOIN vacancies v ON ei.employer_id = v.employer_id
GROUP BY i.name
ORDER BY COUNT(v.id) DESC
LIMIT 10
'''

In [196]:
# результат запроса
df = pd.read_sql_query(query_7_2, connection)
df

,Сфера деятельности,Количество вакансий
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
5,Кадровые агентства,1143
6,Консалтинговые услуги,1137
7,Интернет-провайдер,1095
8,Розничная сеть (продуктовая),1034
9,Фиксированная связь,1028


Таким образом определили ведущие сферы деятельности в области ИТ и регионы (города), в которых они востребованы, которыми являются столичные и крупные города (мегаполисы).

3. Далее проведем анализ количества работодателей по сферам деятельности. Выведем ТОП-10 сфер деятельности в привезке к количеству работодателей.

In [197]:
# текст запроса
query_7_3 = f'''SELECT
    i.name AS "Сфера деятельности",
    COUNT(DISTINCT ei.employer_id) AS "Количество работодателей"
FROM employers_industries ei
-- Связь между работодателями и сферами их деятельности
JOIN industries i ON ei.industry_id = i.id
GROUP BY i.name
ORDER BY COUNT(DISTINCT ei.employer_id) DESC
LIMIT 10
'''

In [198]:
# результат запроса
df = pd.read_sql_query(query_7_3, connection)
df

,Сфера деятельности,Количество работодателей
0,Разработка программного обеспечения,3553
1,"Системная интеграция, автоматизации технологи...",2993
2,"Интернет-компания (поисковики, платежные систе...",1675
3,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798
4,Консалтинговые услуги,662
5,Кадровые агентства,389
6,"Электронно-вычислительная, оптическая, контрол...",374
7,Розничная сеть (продуктовая),368
8,Интернет-магазин,351
9,"Архитектура, проектирование",346


4. Проведем анализ рынка труда в различных городах.

In [199]:
# текст запроса с количеством вакансий более 10
# Рассчет средней з/п по регионам
query_7_4 = f'''SELECT
    a.name AS "Название города (региона)",
    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2)) AS "Средняя з/п", 
    COUNT(v.id) AS "Количество вакансий"
FROM areas a
JOIN vacancies v ON v.area_id = a.id -- Приссоединяем к таблице вакансии таблицу регионы
-- Фильтр на NULL
WHERE (v.salary_to   IS NOT NULL
    OR v.salary_from IS NOT NULL)
GROUP BY 1
-- Фильтр на вакансии более 10
HAVING COUNT(v.id) > 10 
ORDER BY 2 DESC
LIMIT 10
'''
# результат запроса с количеством вакансий более 10
df = pd.read_sql_query(query_7_4, connection)
df

,Название города (региона),Средняя з/п,Количество вакансий
0,Турция,235435.0,19
1,Кипр,223736.0,39
2,Армения,219570.0,106
3,Сербия,208423.0,12
4,Тбилиси,207156.0,80
5,Батуми,199834.0,18
6,Иннополис,175882.0,17
7,Баку,155747.0,51
8,Москва,134140.0,1592
9,Лобня,121136.0,11


Как показывает запрос выше, более высокие средние з/п в городах за пределами России.

Пэтому попробуем увеличить значение количества вакансий до 100.

In [200]:
# текст запроса с количеством вакансий более 100
# Рассчет средней з/п по регионам
query_7_5 = f'''SELECT
    a.name AS "Название города (региона)",
    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2)) AS "Средняя з/п", 
    COUNT(v.id) AS "Количество вакансий"
FROM areas a
JOIN vacancies v ON v.area_id = a.id -- Приссоединяем к таблице вакансии таблицу регионы
-- Фильтр на NULL
WHERE (v.salary_to   IS NOT NULL
    OR v.salary_from IS NOT NULL)
GROUP BY 1
-- Фильтр на вакансии более 100
HAVING COUNT(v.id) > 100 
ORDER BY 2 DESC
LIMIT 10
'''
# результат запроса с количеством вакансий более 10
df = pd.read_sql_query(query_7_5, connection)
df

,Название города (региона),Средняя з/п,Количество вакансий
0,Армения,219570.0,106
1,Москва,134140.0,1592
2,Новосибирск,107777.0,929
3,Санкт-Петербург,104410.0,1203
4,Сочи,99437.0,155
5,Томск,99319.0,350
6,Нижний Новгород,96059.0,680
7,Пермь,95985.0,352
8,Ростов-на-Дону,94065.0,612
9,Калининград,92357.0,181


Армения попапала в этом список ТОП-10. Но теперь стало видно, что средний уровень з/п значительно меньше, что за рубежом почти в 2 раза, однако, стоит отметить, что количество вакансий представленных в России значительно больше, чем за рубежом. Также отметим, что уровень средней з/п выше в более больших городах (мегаполисах).

Этот факт стоит учитывать при более детальном анализе расброса з/п, проводя анализ этого критерия по вакансим в России, т.к. показатели з/п за рубежжом могут негативно сказать на анализе представленных данных и показать неверный результат.

***

# Общий вывод по проекту

In [192]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

# Общий вывод по проекту:

1. База данных содержит 49197 вакансий, в которых представлен 23501 работодатель, содержится информация о 1362 регионах по 294 сферам деятельности.
2. Лидерами по количеству вакансий, з/п являются крупные города (столица и мегаполисы). Пэтому стоит отдельно отметить и выделить их значимость на рынке труда, что в значительной мере в будущем появлияет на развитие инфраструктуры в этих городах, что соответственно повличет за собой и увеличение числа вакансий в сфере ИТ и DS в частности. Т.к. развитие мегаполиса напрямую связано и с развитием ИТ. Поэтому в ближайшее несколько лет стоит ждать увеличения числа вакансий в сфере DS и снижения числа классических ИТ специальностей, т.к. новые технологии будут приходить на смену старым.
3. Компания Яндекс занимает первое место по количеству вакансий - 1933 и это очевидно, т.к. данная компания является одним из флагманов в сфере ИТ и DS в частности. Но в будущем стоит ждать появление конкурентов в указанных областях, т.к. другие компании тоже готовы развивать ИТ.
4. Среднее значение для нижней зарплатной вилки составляет 71065, а для верхней зарплатной вилки - 110537. Опыт работы от 1 года до 3 лет является наиболее востребованным.
5. Полная занятость является наиболее популярным типом занятости в настоящее время. Но возможно и здесь могжно ожидать изменений, т.к. работодатели и соискатели за последнее время получили опыт комьюнити дистанционной работы, поэтому в будущем следует ждать увеличения числа вакансий с типом занятости удаленная или проектная работа.
6. Отдельно отмечаем, что количество вакансий, связанных с работой с данными, составляет 1771. Количество вакансий, где требуются знания языка программирования Python указывается в качестве ключевого навыка, составляет 351. Среднее количество ключевых навыков для дата-сайентиста составляет более шести, среди которых встречаются требования к знанию языка SQL.
7. Отдельным пунктом стоит отметить, что на общем фоне з/п в сфере DS, имеет очень хороший уровень и в обозримом будущем стоит ждать увеличение, т.к. из указанного выше количества предоставлчемых рабочих мест будет увеличиваться и также будет расти конкуренция среди специалистов этого профиля. При необходимости можно еще провести дополнительные исследования в этой области и определить медианные значения з/п.